In [1]:
#todo KNN imputation if the missing rate is high.
#todo Create new features that may capture underlying patterns (e.g., interaction terms, polynomial features).
#todo create separate file for data preparation

In [2]:
##
import pandas as pd
import numpy as np
import warnings

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from TargetEncoder import TargetEncoder
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [3]:
warnings.filterwarnings('ignore')

In [4]:
##
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

In [5]:
##
submission = pd.read_csv('data/sample_submission.csv')
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
print(train.shape)

(750000, 12)


In [6]:
drop_cols = ['id']
target_col = ['Listening_Time_minutes']
cat_cols = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
num_cols = [col for col in train.columns if col not in drop_cols + cat_cols + target_col]

In [7]:
train.shape

(750000, 12)

In [75]:
X = train.drop(drop_cols + target_col, axis=1)
y = train['Listening_Time_minutes']

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Outliers
 

In [10]:
outliers = 'Episode_Length_minutes > 140 or Number_of_Ads > 3'
index_drop = X_train.query(outliers).index
X_train.drop(index_drop, inplace=True)
y_train.drop(index_drop, inplace=True)

# Baseline Model:

In [79]:
model = CatBoostRegressor(random_state=42, cat_features = cat_cols)

In [80]:
summary = model.select_features(X_train, y_train, 
                      eval_set=(X_test, y_test),
                      features_for_select='0-9',
                      num_features_to_select=5,
                      steps=2,
                      train_final_model=False,                            
                      logging_level='Silent')

In [81]:
# Посмотрим на список отобранных фичей (не отранжирован по важности)
print(summary['selected_features_names'])
# И на лучшее значение лосса
print(f"Best loss: {summary['loss_graph']['loss_values'][-1]}")

['Episode_Title', 'Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']
Best loss: 13.026719406495692


In [73]:
# В summary сохраняется полный отчет работы алгоритма
summary

{'selected_features': [0, 1, 2, 3, 4, 5, 20, 29],
 'eliminated_features_names': ['Genre_News',
  'Publication_Day_Sunday',
  'Genre_Health',
  'Genre_Comedy',
  'Publication_Day_Tuesday',
  'Genre_Music',
  'Episode_Sentiment_Negative',
  'Publication_Day_Friday',
  'Publication_Day_Saturday',
  'Genre_Business',
  'Publication_Time_Morning',
  'Publication_Time_Afternoon',
  'Publication_Day_Wednesday',
  'Genre_Lifestyle',
  'Genre_Sports',
  'Episode_Sentiment_Neutral',
  'Genre_True Crime',
  'Genre_Education',
  'Publication_Day_Monday',
  'Publication_Time_Night',
  'Publication_Time_Evening',
  'Genre_Technology'],
 'loss_graph': {'main_indices': [0, 15],
  'removed_features_count': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22],
  'loss_values': [12.984754239775102,
   12.984321046140481,
   12.983997079217131,
   12.98402684389806,
   12.984085474105473,
   12.984152002249521,

# Check model

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

outliers = 'Episode_Length_minutes > 140 or Number_of_Ads > 3'
index_drop = X_train.query(outliers).index
X_train.drop(index_drop, inplace=True)
y_train.drop(index_drop, inplace=True)


# Perform Cross-Validation (5-fold)
model = CatBoostRegressor(random_state=42, cat_features = cat_cols)

cv_rmse = np.sqrt(-cross_val_score(model, X_train, y_train,
                                   scoring="neg_mean_squared_error", cv=5, n_jobs=-1, verbose=100))

# Train on full training set
model.fit(X_train, y_train)

# Make predictions on test set
y_pred = model.predict(X_test)

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print Results
print(f"Cross-Validation RMSE: {cv_rmse.mean():.4f} ± {cv_rmse.std():.4f}")
print(f"Test Set RMSE: {test_rmse:.4f}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  8.4min remaining: 12.6min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  8.5min remaining:  5.6min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  8.5min finished
Learning rate set to 0.112493
0:	learn: 24.9676394	total: 395ms	remaining: 6m 34s
1:	learn: 23.0904796	total: 606ms	remaining: 5m 2s
2:	learn: 21.4830960	total: 806ms	remaining: 4m 27s
3:	learn: 20.1219457	total: 984ms	remaining: 4m 4s
4:	learn: 18.9673658	total: 1.24s	remaining: 4m 6s
5:	learn: 17.9838802	total: 1.43s	remaining: 3m 56s
6:	learn: 17.1641140	total: 1.58s	remaining: 3m 43s
7:	learn: 16.4645547	total: 1.73s	remaining: 3m 35s
8:	learn: 15.8947692	total: 1.9s	remaining: 3m 29s
9:	learn: 15.4126402	total: 2.03s	remaining: 3m 21s
10:	learn: 15.0178227	total: 2.21s	remaining: 3m 18s
11:	learn: 14.6959161	total: 2

# Final model for submission

In [ ]:
# outliers = 'Episode_Length_minutes > 140 or Number_of_Ads > 3'
# index_drop = X.query(outliers).index
# X.drop(index_drop, inplace=True)
# y.drop(index_drop, inplace=True)

In [ ]:
# enc = TargetEncoder()
# X, X_test = target_encoder(X, y, test, ['Podcast_Name', 'Episode_Title'], enc)

In [ ]:
# baseline_pipeline.fit(X, y)

In [ ]:
# # Make predictions on test set
# submission['Listening_Time_minutes'] = baseline_pipeline.predict(X_test)


In [ ]:
# submission.to_csv('submission.csv', index=False)